# **K Nearest Neighbor**


Estimated time needed: **30** minutes


In this lab, you will learn about and practice the K Nearest Neighbor (KNN) model. KNN is a straightforward but very effective model that can be used for both classification and regression tasks. If the feature space is not very large, KNN can be a high-interpretable model because you can explain and understand how a prediction is made by looking at its nearest neighbors.


We will be using a tumor sample dataset containing lab test results about tumor samples. The objective is to classify whether a tumor is malicious (cancer) or benign. As such, it is a typical binary classification task.


## Objectives


After completing this lab, you will be able to:


* Train KNN models with different neighbor hyper-parameters
* Evaluate KNN models on classification tasks
* Tune the number of neighbors and find the optimized one for a specific task


----


First, let's install `seaborn` for visualization tasks and import required libraries for this lab.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.3 numpy==1.21.2 ipywidgets==7.4.2 scipy==7.4.2 tqdm==4.62.3 matplotlib==3.5.0 seaborn==0.9.0
# Note: If your environment doesn't support "!mamba install", use "!pip install".

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
# Evaluation metrics related methods
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Define a random seed to reproduce any random process
rs = 123

In [ ]:
# Ignore any deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Load and explore the tumor sample dataset


We first load the dataset `tumor.csv` as a Pandas dataframe:


In [ ]:
# Read datast in csv format
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/datasets/tumor.csv"
tumor_df = pd.read_csv(dataset_url)

Then, let's quickly take a look at the head of the dataframe.


In [ ]:
tumor_df.head()

And, display its columns.


In [ ]:
tumor_df.columns

Each observation in this dataset contains lab test results about a tumor sample, such as clump or shapes. Based on these lab test results or features, we want to build a classification model to predict if this tumor sample is malicious (cancer) or benign. The target variable `y` is specified in the `Class` column.


Then, let's split the dataset into input `X` and output `y`:


In [ ]:
X = tumor_df.iloc[:, :-1]
y = tumor_df.iloc[:, -1:]

And, we first check the statistics summary of features in `X`:


In [ ]:
X.describe()

As we can see from the above cell output, all features are numeric and ranged between 1 to 10. This is very convenient as we do not need to scale the feature values as they are already in the same range.


Next, let's check the class distribution of output `y`:


In [ ]:
y.value_counts(normalize=True)

In [ ]:
y.value_counts().plot.bar(color=['green', 'red'])

We have about 65% benign tumors (`Class = 0`) and 35% cancerous tumors (`Class = 1`), which is not a very imbalanced class distribution. 


## Process and split training and testing datasets


In [ ]:
# Split 80% as training dataset
# and 20% as testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = rs)

## Train and evaluate a KNN classifier with the number of neighbors set to 2


Training a KNN classifier is very similar to training other classifiers in `sklearn`, we first need to define a `KNeighborsClassifier` object. Here we use `n_neighbors=2` argument to specify how many neighbors will be used for prediction, and we keep other arguments to be their default values.


In [ ]:
# Define a KNN classifier with `n_neighbors=2`
knn_model = KNeighborsClassifier(n_neighbors=2)

Then we can train the model with `X_train` and `y_train`, and we use ravel() method to convert the data frame `y_train` to a vector.


In [ ]:
knn_model.fit(X_train, y_train.values.ravel())

And, we can make predictions on the `X_test` dataframe.


In [ ]:
preds = knn_model.predict(X_test)

To evaluate the KNN classifier, we provide a pre-defined method to return the commonly used evaluation metrics such as accuracy, recall, precision, f1score, and so on, based on the true classes in the 'y_test' and model predictions.


In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp, average='binary')
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

In [ ]:
evaluate_metrics(y_test, preds)

We can see that there is a great classification performance on the tumor sample dataset. This means the KNN model can effectively recognize cancerous tumors.
Next, it's your turn to try a different number of neighbors to see if we could get even better performance.


## Coding exercise: Train and evaluate a KNN classifier with number of neighbors set to 5


First, define a KNN classifier with KNeighborsClassifier class:


In [ ]:
# Type your code here


Then train the model with `X_train` and `y_train`:


In [ ]:
# Type your code here


And, make predictions on `X_test` dataframe:


At last, you can evaluate your KNN model with provided `evaluate_metrics()` method.


<details><summary>Click here for a sample solution</summary>

```python
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)
```

</details>


## Tune the number of neighbors to find the optmized one


OK, you may wonder which `n_neighbors` argument may give you the best classification performance. We can try different `n_neighbors` (the K value) and check which `K` gives the best classification performance.


Here we could try K from 1 to 50, and store the aggregated `f1score` for each k into a list.


In [ ]:
# Try K from 1 to 50
max_k = 50
# Create an empty list to store f1score for each k
f1_scores = []

Then we will train 50 KNN classifiers with K ranged from 1 to 50.


In [ ]:
for k in range(1, max_k + 1):
    # Create a KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    # Train the classifier
    knn = knn.fit(X_train, y_train.values.ravel())
    preds = knn.predict(X_test)
    # Evaluate the classifier with f1score
    f1 = f1_score(preds, y_test)
    f1_scores.append((k, round(f1_score(y_test, preds), 4)))
# Convert the f1score list to a dataframe
f1_results = pd.DataFrame(f1_scores, columns=['K', 'F1 Score'])
f1_results.set_index('K')

This is a long list and different to analysis, so let's visualize the list using a linechart.


In [ ]:
# Plot F1 results
ax = f1_results.plot(figsize=(12, 12))
ax.set(xlabel='Num of Neighbors', ylabel='F1 Score')
ax.set_xticks(range(1, max_k, 2));
plt.ylim((0.85, 1))
plt.title('KNN F1 Score')

As we can see from the F1 score linechart, the best `K` value is 5 with about `0.9691` f1score.


## Next steps


Great! Now you have learned about and applied the KNN model to solve a real-world tumor type classification problem. You also tuned the KNN to find the best K value. Later, you will continue learning other popular classification models with different structures, assumptions, cost functions, and application scenarios.
